# ✅ VRSecretary System Check

This notebook helps you check that **all main services** used by VRSecretary are working:

- `.env` configuration
- **Ollama** (local LLM) – if `MODE=offline_local_ollama`
- **FastAPI gateway** (`/health` and `/api/vr_chat`)
- **Chatterbox TTS** (audio synthesis)
- **watsonx.ai** (indirectly, via the gateway, if `MODE=online_watsonx`)

Run the cells from top to bottom.

> **Tip:** Start this notebook from the **repo root** (where `Makefile` is) with the
> `vrsecretary-env` kernel if you followed the installation guide.


In [ ]:
import os
from pathlib import Path
from textwrap import indent

print("Python version:")
import sys; print(sys.version)

try:
    import requests
    print("✅ `requests` is available.")
except ImportError:
    print("⚠️ `requests` is not installed. HTTP checks will be skipped.")
    requests = None

def find_env_file():
    # Try common locations relative to current working dir
    cwd = Path.cwd()
    candidates = [
        cwd / "backend" / "gateway" / ".env",
        cwd.parent / "backend" / "gateway" / ".env",
    ]
    for p in candidates:
        if p.is_file():
            return p
    return None

env_path = find_env_file()
print("\nLooking for backend .env file...")
if env_path is None:
    print("❌ Could not find backend/gateway/.env relative to this notebook.")
    print("   Make sure you've created it (e.g., from backend/docker/env.example)")
else:
    print(f"✅ Found .env at: {env_path}")


## 1️⃣ Load and show configuration from `.env`

This cell parses the `.env` file that the backend uses and prints the
most important settings: `MODE`, `OLLAMA_BASE_URL`, `CHATTERBOX_URL`,
`WATSONX_*`, etc.


In [ ]:
from typing import Dict

def parse_env(path: Path) -> Dict[str, str]:
    cfg = {}
    if not path or not path.is_file():
        return cfg
    for line in path.read_text(encoding="utf-8").splitlines():
        line = line.strip()
        if not line or line.startswith("#"):
            continue
        if "=" not in line:
            continue
        key, val = line.split("=", 1)
        cfg[key.strip()] = val.strip()
    return cfg

env_cfg = parse_env(env_path) if env_path else {}

if not env_cfg:
    print("⚠️ No configuration loaded (missing or empty .env). Many tests will be skipped.")
else:
    keys_of_interest = [
        "MODE",
        "OLLAMA_BASE_URL",
        "OLLAMA_MODEL",
        "OLLAMA_TIMEOUT",
        "CHATTERBOX_URL",
        "CHATTERBOX_TIMEOUT",
        "WATSONX_URL",
        "WATSONX_PROJECT_ID",
        "WATSONX_MODEL_ID",
        "WATSONX_API_KEY",
        "WATSONX_TIMEOUT",
        "SESSION_MAX_HISTORY",
    ]
    print("Loaded configuration from .env:\n")
    for k in keys_of_interest:
        if k in env_cfg:
            val = env_cfg[k]
            if k.endswith("API_KEY"):
                if len(val) > 6:
                    val = val[:3] + "***" + val[-3:]
                else:
                    val = "***"  # too short, mask entirely
            print(f"  {k:20s} = {val}")

mode = env_cfg.get("MODE", "offline_local_ollama")
print(f"\nActive MODE according to .env: {mode}")
if mode == "offline_local_ollama":
    print("→ Expecting a local Ollama server.")
elif mode == "online_watsonx":
    print("→ Expecting IBM watsonx.ai to be configured.")
else:
    print("→ MODE is custom or unrecognized; tests will still try to run.")


## 2️⃣ Test Ollama (if configured)

This cell tries to:

1. Import the `ollama` Python client.
2. Pull a small model (by default `qwen2.5:0.5b-instruct` or `OLLAMA_MODEL`).
3. Run a tiny chat request.

If `MODE=offline_local_ollama` this is the primary LLM used by the backend.


In [ ]:
ollama_base = env_cfg.get("OLLAMA_BASE_URL", "http://localhost:11434")
ollama_model = env_cfg.get("OLLAMA_MODEL", "qwen2.5:0.5b-instruct")

print(f"Ollama base URL: {ollama_base}")
print(f"Ollama model   : {ollama_model}\n")

try:
    import ollama
    print("✅ Python client `ollama` is available.")
except Exception as e:
    print("⚠️ Python package `ollama` is not available. Install it with `pip install ollama`.")
    ollama = None

if ollama is not None:
    try:
        print(f"📥 Pulling/checking model: {ollama_model} ...")
        ollama.pull(ollama_model)
        print(f"✅ Model ready: {ollama_model}\n")
    except Exception as e:
        print("❌ Could not pull the model. Is the Ollama server running at", ollama_base, "?")
        raise

    print("Running a tiny chat test...\n")
    try:
        resp = ollama.chat(
            model=ollama_model,
            messages=[{
                'role': 'user',
                'content': "Di' solo 'Ciao!' in italiano e poi 1 consiglio per studiare meglio.",
            }],
        )
        print(resp['message']['content'])
        print("\n✅ Ollama chat completed.")
    except Exception as e:
        print("❌ Ollama chat failed:", e)
        raise
else:
    print("Skipping Ollama test because Python client is missing.")


## 3️⃣ Test FastAPI Gateway (`/health` and `/api/vr_chat`)

This step assumes you have already started the backend, e.g.:

```bash
make run-gateway
```

or manually:

```bash
cd backend/gateway
uvicorn vrsecretary_gateway.main:app --host 0.0.0.0 --port 8000
```


In [ ]:
if requests is None:
    print("⚠️ Skipping HTTP tests because `requests` is not installed.")
else:
    gateway_host = env_cfg.get("GATEWAY_HOST", "http://localhost:8000")
    if not gateway_host.startswith("http"):
        gateway_host = "http://" + gateway_host
    print("Gateway base URL:", gateway_host)

    # Health check
    try:
        print("\nChecking /health ...")
        r = requests.get(gateway_host.rstrip('/') + "/health", timeout=5)
        print("Status:", r.status_code)
        print("Body  :", r.text)
        if r.ok:
            print("✅ Gateway /health OK.")
        else:
            print("❌ Gateway /health returned non-OK status.")
    except Exception as e:
        print("❌ Failed to reach /health:", e)

    # Simple /api/vr_chat test
    try:
        import uuid
        session_id = "test-" + str(uuid.uuid4())[:8]
        print("\nSending test chat to /api/vr_chat ...")
        payload = {
            "session_id": session_id,
            "user_text": "Ciao Ailey! Chi sei? Rispondi in una frase.",
        }
        r = requests.post(
            gateway_host.rstrip('/') + "/api/vr_chat",
            json=payload,
            timeout=60,
        )
        print("Status:", r.status_code)
        if r.ok:
            data = r.json()
            print("assistant_text:\n", data.get("assistant_text"))
            audio_b64 = data.get("audio_wav_base64")
            if audio_b64:
                print("\nAudio length (base64 chars):", len(audio_b64))
                print("✅ Gateway returned text AND audio.")
            else:
                print("\n⚠️ audio_wav_base64 is empty. LLM works, but TTS might be disabled.")
            print("\n✅ /api/vr_chat test completed.")
        else:
            print("❌ /api/vr_chat returned non-OK status.")
            print(r.text)
    except Exception as e:
        print("❌ Failed to call /api/vr_chat:", e)


## 4️⃣ Test Chatterbox TTS directly

This checks that the TTS server is reachable and returns audio. It does **not**
play the sound, only confirms that bytes are returned.


In [ ]:
if requests is None:
    print("⚠️ Skipping Chatterbox test because `requests` is not installed.")
else:
    chatter_url = env_cfg.get("CHATTERBOX_URL", "http://localhost:4123")
    if not chatter_url.startswith("http"):
        chatter_url = "http://" + chatter_url
    endpoint = chatter_url.rstrip('/') + "/v1/audio/speech"
    print("Chatterbox endpoint:", endpoint)

    try:
        payload = {
            "input": "Ciao, sono Ailey, la tua segretaria VR.",
            "temperature": 0.6,
            "cfg_weight": 0.5,
            "exaggeration": 0.35,
        }
        r = requests.post(endpoint, json=payload, timeout=60)
        print("Status:", r.status_code)
        if r.ok:
            audio_bytes = r.content
            print("Received", len(audio_bytes), "bytes of WAV data.")
            if len(audio_bytes) > 0:
                print("✅ Chatterbox seems to be working.")
            else:
                print("⚠️ Response OK but empty body.")
        else:
            print("❌ Chatterbox returned non-OK status:")
            print(r.text)
    except Exception as e:
        print("❌ Failed to call Chatterbox:", e)


## 5️⃣ Summary

If all the sections above printed green checkmarks (✅), your VRSecretary
backend and its dependencies are healthy.

You can now:

- Run the Unreal demo project and talk to Ailey, or
- Use the same `/api/vr_chat` endpoint from other engines (e.g., Unity).
